In [1]:
### same to "Batch_M1_0506"
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib as jl
import cebra.datasets
from cebra import CEBRA
import scipy.io as sio
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
import sklearn.metrics
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") ### select either 0 or 1
print(f'Using device: {device}')

Using device: cuda:0


In [2]:
dur = 40
iterations = 5000
output_dimension = 3
cebra_veldir_model = CEBRA(model_architecture='offset1-model',
                           batch_size=512,
                           learning_rate = 0.0001,
                           temperature = 1,
                           output_dimension = output_dimension,
                           max_iterations=iterations,
                           distance='cosine',
                           conditional='time_delta',
                           device="cuda:1", ### 'cuda_if_available'
                           verbose=True,
                           time_offsets=1)
def split_data(neural, continuous_index, train_trial):
            split_idx = train_trial*dur 
            neural_train = neural[:split_idx]
            neural_test = neural[split_idx:]
            continuous_index_train = continuous_index[:split_idx]
            continuous_index_test = continuous_index[split_idx:]
            return neural_train,neural_test,continuous_index_train,continuous_index_test
        
angle_to_new_value = {-180: 4,-135: 5,-90: 6,-45: 7,0: 0,45: 1,90: 2,135: 3,180: 4}

directory = "./data/SU_16M1/"
files = os.listdir(directory)
for file in files:
    if "Chewie" in file: ### "Chewie" or "Mihili" in file 
        mat_contents = sio.loadmat(os.path.join(directory, file))
        filename_parts = file.split("_neural_con_dis_index")
        new_filename = filename_parts[0] + "_embed_"+str(iterations)+"itr_M1.npz"
        file_save = os.path.join(directory, new_filename)
        print(file_save)

        neural = mat_contents['neural_M1']
        continuous_index_2d = mat_contents['continuous_index']*10
        discrete_index = mat_contents['discrete_index']
        
        vectorized_map = np.vectorize(lambda x: angle_to_new_value[x])
        discrete_index = vectorized_map(discrete_index)
        continuous_index = np.hstack((continuous_index_2d, discrete_index*45))
        total_trial = int(discrete_index.shape[0]/dur)
        train_trial = int(total_trial*0.8)
        test_trial = total_trial-train_trial
        
        neural_train, neural_test, continuous_index_train, continuous_index_test = split_data(neural, continuous_index, train_trial)
        target_angle_train = continuous_index_train[:, 2].copy()
        target_angle_test = continuous_index_test[:, 2].copy()
        cebra_veldir_model.fit(neural_train, continuous_index_train)
        cebra_veldir_train = cebra_veldir_model.transform(neural_train)
        cebra_veldir_test  = cebra_veldir_model.transform(neural_test)
 
        train_loss = cebra_veldir_model.state_dict_['loss']
#         train_loss_stable = train_loss[-10:].numpy()
#         fig = plt.figure(figsize=(6,5))
#         ax = plt.subplot(111)
#         ax.plot(train_loss, c='deepskyblue')
#         ax.spines['top'].set_visible(False)
#         ax.spines['right'].set_visible(False)
#         ax.set_xlabel('Iterations')
#         ax.set_ylabel('RnC Loss')
#         plt.legend(bbox_to_anchor=(0.5,0.3), frameon = False )
#         plt.title('iterations='+str(iterations)+'  final loss='+str(np.mean(train_loss_stable)))
#         new_filename = filename_parts[0] + "_train_loss_"+str(iterations)+"itr.pdf" 
#         output_path = os.path.join(directory, new_filename)
#         plt.savefig(output_path)
#         plt.close(fig)

        np.savez(file_save,
                 train_loss_stable=train_loss,
                 cebra_veldir_train=cebra_veldir_train,
                 cebra_veldir_test=cebra_veldir_test,
                 continuous_index_train=continuous_index_train,
                 continuous_index_test=continuous_index_test)

./data/Chewie_20161014_embed_5000itr_M1.npz


pos: -0.8945 neg:  6.4037 total:  5.5091 temperature:  1.0000: 100%|█| 5000/5000
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
